In [1]:
# checking to make sure kernal is within virtual environment 
import sys
import site
print(site.getsitepackages())

['/Users/krutheekarajkumar/PycharmProjects/Proj1venv/lib/python3.8/site-packages']


In [2]:
import requests
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
import re
import csv

#Natural Language Processing Packages
import re
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer

In [3]:
# List of commonly occuring stop words from http://www.nltk.org/nltk_data/
stopwords = open("english", "r")
stopwords = [word for word in stopwords]

In [4]:
# works!
def extract_job_title_from_result(soup):
    jobs = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
            jobs.append(a["title"])
    return(jobs)

In [5]:
# companies work, and the list of IDs also work
def extract_company_from_result(soup):
    companies = []
    recJobLoc = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        jobrecloc = div.find_all(name="div", attrs = {"class":"recJobLoc"})
        for jobid in jobrecloc:
            recJobLoc.append(jobid.get("id").replace("recJobLoc_", ""))
        company = div.find_all(name="span", attrs = {"class":"company"})
        if len(company) > 0:
            for b in company:
                companies.append(b.text.strip())
        else:
            sec_try = div.find_all(name="span", attrs = {"class":"result - link - source"})
            for span in sec_try:
                companies.append(span.text.strip())
    return (companies, recJobLoc)

In [6]:
def DECOMISSIONED_extract_location_from_result(soup):
    locations = []
    spans = soup.findAll("span", attrs={"class": "location"})
    for span in spans:
        locations.append(span.text)
    return(locations)

In [7]:
def isQualification(text):
    tokenizer = RegexpTokenizer(r'\w+') 
    porter = PorterStemmer()
    keywords = ['experi','requir','qualif','skill', 'educ']
    # Cleaning text
    clean_text = []
    new_text = re.sub('\n', '', text)
    new_text = re.sub('%', '', new_text)
    new_text = re.sub('@', '', new_text)
    new_text = re.sub(r'[0-9]', '', new_text)
    new_text = new_text.lower()
    new_text = tokenizer.tokenize(new_text)
    #print(new_text)
    for nt in new_text: 
        if nt in stopwords:
            pass
        else: 
            clean_text.append(nt)
            #stemmedword.append(porter.stem(nt))
            if porter.stem(nt) in keywords:
                return True

In [59]:
def DECOMMISIONED_extract_salary_from_result(soup):
    salaries = []
    desiredQualifications = []
    desiredQualificationsDescription = []
    
    for headers in soup.find_all(name='h1'):
        if isQualification(headers.text):
            desiredQualifications.append(headers.parent.next_sibling.text)
    for headers in soup.find_all(name='h2'):
        if isQualification(headers.text):
            desiredQualifications.append(headers.parent.next_sibling.text)
    for headers in soup.find_all(name='h3'):
        if isQualification(headers.text):
            desiredQualifications.append(headers.parent.next_sibling.text)
    
    """for div in soup.find_all(name="div", attrs={"id":"jobDescriptionText"}):
        for item in div.find_all('b'):
            if isQualification(item.text):
                for tag in item.next_elements:
                    print("NAME:", tag.name)
                    print("TAG: ", tag)"""
                    
    for div in soup.find_all(name="div", attrs={"id":"jobDescriptionText"}):
        for tag in div.next_elements:
            if (tag.name == 'b') and isQualification(tag.text) == True:
                for t in tag.next_elements:
                    if  t.name == 'script' or t.name == 'span':
                        break 
                    else:
                        try:
                            desiredQualificationsDescription.append(t.text)
                        except: 
                            pass
            else: 
                pass
        
   
                
                #if isQualification(item.text) == True:
                #    print("HERE: ", item.text)
                #    print("***********")
                    #for tag in item.next_elements:
                    #    print("NAME:", tag.name)
                    #    print("TAG: ", tag)
    

                #print([tag.name if tag.name else tag for tag in item.parent.next_elements])
                #desiredQualificationsDescription.append(item.parent.next_sibling.text)
            #salaries.append(item.text)
        """try:
        
            salaries.append(div.find("nobr").text)
        except:
            try:
                div_two = div.find(name="div", attrs={"class":"salaryText"})
                div_three = div_two.find("div")
                salaries.append(div_three.text.strip())
            except:
                salaries.append("Nothing_found")
                 if len(desiredQualificationsDescription) == 0 or desiredQualificationsDescription[0] == "" : 
        for div in soup.find_all(name="div", attrs={"id":"jobDescriptionText"}):
            for i in div.find_all('div', string=re.compile("requir")):
                    print("****")
                    print(i)"""
    desiredQualifications.extend(desiredQualificationsDescription)
    if len(desiredQualifications) >= len(desiredQualificationsDescription):
        return (desiredQualifications)
    else:
        return(desiredQualificationsDescription)
    print(desiredQualifications)
    #return(desiredQualifications)#, salaries)

In [151]:
def extract_job_requirements_from_result(soup):
    salaries = []
    desiredQualifications = []
    desiredQualificationsDescription = []
    desiredQualificationslists = []
    
    if len(desiredQualifications) == 0:
        for headers in soup.find_all(name='h1'):
            if isQualification(headers.text):
                desiredQualifications.append(headers.parent.next_sibling.text)
    elif len(desiredQualifications) == 0:           
        for headers in soup.find_all(name='h2'):
            if isQualification(headers.text):
                desiredQualifications.append(headers.parent.next_sibling.text)
    else:
        for headers in soup.find_all(name='h3'):
            if isQualification(headers.text):
                desiredQualifications.append(headers.parent.next_sibling.text)

    for div in soup.find_all(name="div", attrs={"id":"jobDescriptionText"}):
        for item in div.find_all('b'):
            if isQualification(item.text):
                for tag in item.parent.next_siblings:
                    try:
                        desiredQualificationsDescription.append(tag.text)
                    except:
                        pass
                    
    if len(desiredQualifications) < 50 and len(desiredQualificationsDescription)< 50:
        for div in soup.find_all(name="div", attrs={"id":"jobDescriptionText"}):
            for item in div.find_all("li"):
                if isQualification(item.text):
                    desiredQualificationslists.append(item.text)
        return(desiredQualificationslists)
    
    else:
        desiredQualifications.extend(desiredQualificationsDescription)
        if len(desiredQualifications) >= len(desiredQualificationsDescription):
            return (desiredQualifications)
        else:
            return(desiredQualificationsDescription)
        print(desiredQualifications)
    #return(desiredQualifications)#, salaries)

In [193]:
def extract_salary_from_result(soup):
    salary = "None found"
    for div in soup.find_all(name="div", attrs={"id":"jobDescriptionText"}):
        for item in div.find_all("p"):
            if re.search('\$\d+', item.text):
                salary = item.text
    
    if re.search('\$\d+', salary) == False:
        for item in div.find_all("b"):
            if re.search('\$\d+', item.text):
                salary = item.text
                
    if re.search('\$\d+', salary) == False:
        for item in div.find_all("span"):
            if re.search('\$\d+', item.text):
                salary = item.text
    
    if re.search('\$\d+', salary) == False:
        for item in div.find_all("il"):
            if re.search('\$\d+', item.text):
                salary = item.text
     
    return salary
            

In [237]:
def extract_job_locations_from_result(soup):
    div = soup.find(name="div", attrs={"class":"jobsearch-InlineCompanyRating"})
    return (div.text)
        

In [238]:
extract_job_locations_from_result(soup1)

'Larson Design Group16 reviews-Beaver, PA 15009'

In [11]:
titlesOfInterest = []
URL = "https://www.indeed.com/jobs?q=data+engineer+%2420%2C000&l=Toronto&start=10"

page = requests.get(URL)
soup = BeautifulSoup(page.text, "html.parser")

In [239]:
#print(extract_job_title_from_result(soup))
#print(extract_company_from_result(soup))
#print(extract_location_from_result(soup))
companyNames, indeedIdentification = extract_company_from_result(soup)
qualifications = []
for i in range(len(companyNames)):
    cmpName = companyNames[i].replace(" ", "%20")
    cmpName = cmpName.replace("'", "%27")
    url1 = (f'https://www.indeed.com/viewjob?cmp={cmpName}&jk={indeedIdentification[i]}')
    #url1 = 'https://www.indeed.com/viewjob?cmp=Monaloh%20Basin%20Engineers&jk=37f74a32956e413f'
    print(url1)
    page = requests.get(url1)
    soup1 = BeautifulSoup(page.text, "html.parser")
    try:
        qualifications.append(extract_job_requirements_from_result(soup1))
        print(extract_job_locations_from_result(soup1))
        #print(extract_salary_from_result(soup1))
    except:
        pass
    

https://www.indeed.com/viewjob?cmp=LANXESS&jk=d1278daae9bfc93d
LANXESS294 reviews-Pittsburgh, PA 15275
https://www.indeed.com/viewjob?cmp=Kinetic%20Personnel%20Group%20Inc.&jk=f1de9c05321e8f24
Kinetic Personnel Group Inc.11 reviews-United States
https://www.indeed.com/viewjob?cmp=Eaton&jk=6905fadd4e4f978c
Eaton4,338 reviews-Township of Moon, PA
https://www.indeed.com/viewjob?cmp=Fidelity%20National%20Financial&jk=af747db88947fba8
Fidelity National Financial682 reviews-Township of Moon, PA
https://www.indeed.com/viewjob?cmp=EY&jk=146a36cd2c632f99
EY7,618 reviews-United States
https://www.indeed.com/viewjob?cmp=CleanAir%20Engineering,%20Inc.&jk=8997fef184366b2a
CleanAir Engineering, Inc.-Pittsburgh, PA 15275
https://www.indeed.com/viewjob?cmp=ITI%20Data&jk=4995af38496c8840
ITI Data-United States
https://www.indeed.com/viewjob?cmp=Mitigation%20Resources%20of%20North%20America&jk=c97056b340f99346
Mitigation Resources of North America-United States
https://www.indeed.com/viewjob?cmp=PrologM

In [240]:
qualifications[3]

['Excellent analytical skills in the analysis of technical problems\n',
 'Excellent communication skills in contact with carriers, colleagues and customers in the trouble shooting process as well as the willingness to work in teams\n',
 'Direct the Process and Logistic Team in coordinating and performing tasks required for successful completion of projects, complex workflows, and troubles.\n',
 'Performing disaster recovery operations and data backups when required.\n',
 'Replacing faulty network hardware components when required.\n',
 '2+ years of experience as a Network Engineer and 2 years with Switching.\n',
 'Experience with enterprise-level network design, engineering, and administration.\n',
 'Experience with common networking devices such as firewalls, switches, and fabric interconnects. Commonly used brands include VeloCloud, Arista, Cisco, Meraki, Aruba, Juniper and Palo Alto.\n',
 'Experience interfacing with customer POCs to evaluate networking needs or requirements and exp

In [253]:
def dataIntakeClean(text):
    singleEntry = ""
    for t in text:
        t = t.replace("'","")
        t = t.replace("\n","")
        singleEntry+=t
    return (singleEntry)

In [254]:
dataIntakeClean(qualifications[3])

'Excellent analytical skills in the analysis of technical problemsExcellent communication skills in contact with carriers, colleagues and customers in the trouble shooting process as well as the willingness to work in teamsDirect the Process and Logistic Team in coordinating and performing tasks required for successful completion of projects, complex workflows, and troubles.Performing disaster recovery operations and data backups when required.Replacing faulty network hardware components when required.2+ years of experience as a Network Engineer and 2 years with Switching.Experience with enterprise-level network design, engineering, and administration.Experience with common networking devices such as firewalls, switches, and fabric interconnects. Commonly used brands include VeloCloud, Arista, Cisco, Meraki, Aruba, Juniper and Palo Alto.Experience interfacing with customer POCs to evaluate networking needs or requirements and explain technical terms to non-technical users or stakeholde

In [16]:
# Do not rewrite the .csv file 
"""with open('qualification.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["qualifications"])
    for t in titlesOfInterest:
        writer.writerow([t])"""

'with open(\'qualification.csv\', \'w\', newline=\'\') as file:\n    writer = csv.writer(file)\n    writer.writerow(["qualifications"])\n    for t in titlesOfInterest:\n        writer.writerow([t])'

### Adding to postgre